In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

In [ ]:
%cd /content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets
path = "/content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets"

In [ ]:
# all library imports here
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
a_file = open("index_dictionary.pkl", "rb")
index_dictionary = pickle.load(a_file)
print(index_dictionary)

In [ ]:
import random

def getList(dict): 
    return list(dict.keys()) 

       
keyList = getList(index_dictionary)
print(type(keyList))

random_feature = [] 

for i in range(4):
  random_feature.append(random.sample(keyList, 5))

print(random_feature )

In [ ]:
# load dataset
Xa_train, Xa_test, ya_train, ya_test = np.load("xa_train.npy"), np.load("xa_test.npy"), np.load("ya_train.npy"), np.load("ya_test.npy")
Xc_train, Xc_test, yc_train, yc_test = np.load("xc_train.npy"), np.load("xc_test.npy"), np.load("yc_train.npy"), np.load("yc_test.npy")
Xd_train, Xd_test, yd_train, yd_test = np.load("xd_train.npy"), np.load("xd_test.npy"), np.load("yd_train.npy"), np.load("yd_test.npy")

#pa_test = np.concatenate((pa_test, dt), axis=1)
for feature_groups in random_feature:
  #feature_groups = random_feature[0] 
  no_of_features = 0
  for i in feature_groups:
    no_of_features = no_of_features + index_dictionary[i][1]-index_dictionary[i][0]+1
  print(no_of_features)

  xa_train_n, xa_test_n, ya_train_n, ya_test_n = np.zeros((Xa_train.shape[0],no_of_features)), np.zeros((Xa_test.shape[0],no_of_features)), ya_train, ya_test
  starting_index = 0
  for i in feature_groups:
    ending_index = starting_index + index_dictionary[i][1]-index_dictionary[i][0]
    print(starting_index,ending_index)
    print(index_dictionary[i][1]+1, index_dictionary[i][0])
    xa_train_n[:,starting_index:ending_index+1] = Xa_train[:,index_dictionary[i][0]:index_dictionary[i][1]+1]
    xa_test_n[:,starting_index:ending_index+1] = Xa_test[:,index_dictionary[i][0]:index_dictionary[i][1]+1]
    starting_index = ending_index+1
  print(starting_index)

In [ ]:
#pa_test = np.concatenate((pa_test, dt), axis=1)
xa_trains = []
for i, feature_groups in enumerate(random_feature):
  #no_of_features = 0
  #for j in feature_groups:
   # no_of_features = no_of_features + index_dictionary[j][1]-index_dictionary[j][0]+1
  #print(no_of_features)
  xa_train_temp = np.concatenate([Xa_train[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xa_trains.append(xa_train_temp)
  print(xa_trains[i].shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import pickle
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from tabulate import tabulate
from IPython.display import clear_output

In [ ]:
def avg(avg_list):
  return sum(avg_list)/len(avg_list)

In [ ]:
# for SVM
cv = StratifiedKFold( n_splits=5, shuffle=True)
svc = SVC() # Support vector classifier(SVC)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []

svc_rows = []
headers = ["Random-Set", "Classifier", "Train-Accuracy", "Test-Accuracy", "Train-Specificity", "Test-Specificity", "Train-Sensitivity", "Test-Sensitivity", "Train-MCC", "Test-MCC"]

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      svc.fit(x_train, y_train.ravel())
      test_pred = svc.predict(x_test)
      train_pred = svc.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  svc_rows.append(["Set0"+str(i+1), "SVM", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])
clear_output()

In [ ]:
###  for LogisticRegression

logisticRegression = LogisticRegression(max_iter=500)
#cv = StratifiedKFold( n_splits=5, shuffle=True)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []

lg_rows = []

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      logisticRegression.fit(x_train, y_train.ravel())
      test_pred = logisticRegression.predict(x_test)
      train_pred = logisticRegression.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  lg_rows.append(["Set0"+str(i+1), "LogisticRegression", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),

                  avg(train_mcc), avg(test_mcc)])
  
clear_output()

In [ ]:
###  for XGBClassifier

xgbC = XGBClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
xgbc_rows = []
for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      xgbC.fit(x_train, y_train.ravel())
      test_pred = xgbC.predict(x_test)
      train_pred = xgbC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  xgbc_rows.append(["Set0"+str(i+1), "XGB", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

clear_output()

In [ ]:
###  for AdaBoostClassifier

adaBC = AdaBoostClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
adabc_rows = []

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      adaBC.fit(x_train, y_train.ravel())
      test_pred = adaBC.predict(x_test)
      train_pred = adaBC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  adabc_rows.append(["Set0"+str(i+1),"AdaBoost", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

clear_output()

In [ ]:
###  for RandomForestClassifier

rfC = RandomForestClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
rfc_rows = []

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      rfC.fit(x_train, y_train.ravel())
      test_pred = rfC.predict(x_test)
      train_pred = rfC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  rfc_rows.append(["Set0"+str(i+1),"RandomForest", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

clear_output()